In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Import Necessary Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import random
import cv2
from skimage.metrics import structural_similarity as ssim

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define paths to the original and test images
original_house = '/content/house.png'
original_clock = '/content/clock.png'

test_house = '/content/house_test.png'
test_clock = '/content/clock_test.png'

# Ensure the images exist
required_images = [original_house, original_clock, test_house, test_clock]
for img_path in required_images:
    if not os.path.exists(img_path):
        print(f"Error: {img_path} does not exist. Please upload the image to the specified path.")
        from google.colab import files
        files.upload()
        break

# Directory to save augmented images
augmented_dir = '/content/augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Parameters
img_width, img_height = 224, 224  # EfficientNetB0 input size
batch_size = 32
num_augmented = 850  # Number of augmented images per original image

# Create ImageDataGenerator instances for healthy and impaired augmentations
healthy_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.15,
    brightness_range=[0.8,1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

impaired_datagen = ImageDataGenerator(
    rotation_range=45,  # More rotation to simulate impairments
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.5,1.5],
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to augment images
def augment_image(image_path, label, datagen, save_dir, prefix, num_augmented):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    aug_iter = datagen.flow(
        x,
        batch_size=1,
        save_to_dir=save_dir,
        save_prefix=prefix,
        save_format='png'
    )

    labels = []
    for i in range(num_augmented):
        next(aug_iter)
        labels.append(label)
    return labels

# Create directories for healthy and impaired images
healthy_dir = os.path.join(augmented_dir, 'healthy')
impaired_dir = os.path.join(augmented_dir, 'impaired')
os.makedirs(healthy_dir, exist_ok=True)
os.makedirs(impaired_dir, exist_ok=True)

# Augment healthy images
labels_healthy = []
labels_healthy += augment_image(original_house, 0, healthy_datagen, healthy_dir, 'house_healthy', num_augmented)
labels_healthy += augment_image(original_clock, 0, healthy_datagen, healthy_dir, 'clock_healthy', num_augmented)

# Augment impaired images
labels_impaired = []
labels_impaired += augment_image(original_house, 1, impaired_datagen, impaired_dir, 'house_impaired', num_augmented)
labels_impaired += augment_image(original_clock, 1, impaired_datagen, impaired_dir, 'clock_impaired', num_augmented)

# Combine labels
labels = labels_healthy + labels_impaired

# Collect image file paths and corresponding labels
image_paths = []
for fname in os.listdir(healthy_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(healthy_dir, fname))
for fname in os.listdir(impaired_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(impaired_dir, fname))

# Shuffle the dataset
combined = list(zip(image_paths, labels))
random.shuffle(combined)
image_paths, labels = zip(*combined)

image_paths = np.array(image_paths)
labels = np.array(labels)

# Map numerical labels to string labels
label_mapping = {0: 'Healthy', 1: 'Impaired'}
labels_str = np.array([label_mapping[label] for label in labels])

# Create a dataframe with string labels
full_df = pd.DataFrame({'filename': image_paths, 'class': labels_str})

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    full_df['filename'], full_df['class'], test_size=0.15, random_state=42, stratify=full_df['class'])

# Create separate dataframes for training and validation
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
val_df = pd.DataFrame({'filename': X_val, 'class': y_val})

print(f'Training samples: {len(train_df)}')
print(f'Validation samples: {len(val_df)}')

# Define data generators with preprocessing
train_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

# Create generators
train_generator = train_datagen_flow.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen_flow.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)

# Build an Advanced CNN Model Using Transfer Learning (Functional API)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    '/content/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Train the model
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Fine-Tune the Model
base_model.trainable = True

# Freeze all layers except the top 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the number of fine-tuning epochs
fine_tune_epochs = 10
initial_epoch = len(history.history['loss'])
total_epochs = initial_epoch + fine_tune_epochs

# Continue training with fine-tuning
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=initial_epoch,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Load the best model saved during training
best_model = tf.keras.models.load_model('/content/best_model.keras')

# Function to preprocess and predict a single image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Batch dimension
    return img_array

def predict_image(image_path, model):
    preprocessed = preprocess_image(image_path)
    prediction = model.predict(preprocessed)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = 'Healthy' if class_idx == 0 else 'Impaired'
    confidence = prediction[0][class_idx]
    return class_label, confidence, prediction

# Define test images
test_images = [test_house, test_clock]

# Function to calculate similarity metrics
def calculate_similarity(original_path, test_path):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Convert to grayscale for SSIM
    original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    test_gray = cv2.cvtColor(test, cv2.COLOR_RGB2GRAY)

    # Calculate SSIM
    ssim_score = ssim(original_gray, test_gray)

    # Calculate MSE
    mse_score = np.mean((original - test) ** 2)

    return ssim_score, mse_score

# Function to visualize comparison
def visualize_comparison(original_path, test_path, title):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Calculate difference
    difference = cv2.absdiff(original, test)

    # Convert to grayscale
    difference_gray = cv2.cvtColor(difference, cv2.COLOR_RGB2GRAY)

    # Apply threshold to highlight differences
    _, thresh = cv2.threshold(difference_gray, 30, 255, cv2.THRESH_BINARY)

    # Calculate SSIM for display
    ssim_score, _ = calculate_similarity(original_path, test_path)

    # Plotting
    plt.figure(figsize=(15,5))

    plt.subplot(1,3,1)
    plt.imshow(original.astype('uint8'))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(test.astype('uint8'))
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(thresh, cmap='gray')
    plt.title(f'Difference Map\nSSIM: {ssim_score:.4f}')
    plt.axis('off')

    plt.suptitle(title, fontsize=16)
    plt.show()

# Function to generate Grad-CAM heatmap
def get_gradcam_heatmap(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Get the last convolutional layer
    last_conv_layer = model.get_layer('efficientnetb0').get_layer('top_conv')

    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = Model([model.inputs], [last_conv_layer.output, model.output])

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    # Compute gradients
    grads = tape.gradient(loss, conv_outputs)

    # Compute guided gradients
    guided_grads = grads[0]

    # Weigh the outputs of the conv layer with the gradients
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)

    # Apply ReLU to the heatmap
    heatmap = np.maximum(cam, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap.numpy()

def display_gradcam(image_path, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    # Load the original image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    # Superimpose the heatmap on the image
    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)

    # Convert BGR to RGB for displaying
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Function to generate analysis report
def generate_analysis_report(original_path, test_path, model):
    # Determine the drawing type
    if 'house' in test_path.lower():
        title = 'House Drawing Analysis'
    elif 'clock' in test_path.lower():
        title = 'Clock Drawing Analysis'
    else:
        title = 'Drawing Analysis'

    # Predict label and confidence
    label, conf, pred = predict_image(test_path, model)

    # Calculate similarity metrics
    ssim_score, mse_score = calculate_similarity(original_path, test_path)

    # Display similarity metrics
    print(f'--- {title} ---')
    print(f'Predicted Label: {label} with confidence {conf:.2f}')
    print(f'SSIM Score: {ssim_score:.4f}')
    print(f'MSE Score: {mse_score:.2f}')
    print('\n')

    # Visualize comparison
    visualize_comparison(original_path, test_path, title)

    # Generate Grad-CAM heatmap
    try:
        heatmap = get_gradcam_heatmap(model, test_path)
        print('Grad-CAM Heatmap:')
        display_gradcam(test_path, heatmap)
    except Exception as e:
        print(f"Error generating Grad-CAM: {e}")

    # Provide textual analysis
    if label == 'Impaired':
        print("Analysis: The drawing shows significant deviations from the original, indicating possible cognitive impairments associated with Alzheimer's disease.")
    else:
        print("Analysis: The drawing closely resembles the original, suggesting no significant cognitive impairments detected.")
    print('\n' + '='*80 + '\n')

# Generate analysis reports for test images
for test_img in test_images:
    if 'house' in test_img.lower():
        original_img = original_house
    elif 'clock' in test_img.lower():
        original_img = original_clock
    else:
        continue

    generate_analysis_report(original_img, test_img, best_model)

# Plot Training and Validation Metrics
def plot_training_history(history, fine_tune_history=None):
    plt.figure(figsize=(14,6))

    # Plot Accuracy
    plt.subplot(1,2,1)
    epochs_range = range(len(history.history['accuracy']))
    plt.plot(epochs_range, history.history['accuracy'], label='Train Accuracy')
    plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')

    if fine_tune_history:
        fine_tune_epochs_range = range(len(history.history['accuracy']), len(history.history['accuracy']) + len(fine_tune_history.history['accuracy']))
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['accuracy'], label='Fine-Tune Train Accuracy')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_accuracy'], label='Fine-Tune Validation Accuracy')

    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1,2,2)
    plt.plot(epochs_range, history.history['loss'], label='Train Loss')
    plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')

    if fine_tune_history:
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['loss'], label='Fine-Tune Train Loss')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_loss'], label='Fine-Tune Validation Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Plot the training history
plot_training_history(history, history_fine)


In [ ]:
# Mount Google Drive (optional, if you're saving or loading files from Google Drive)
from google.colab import drive
drive.mount('/content/drive')

# Import Necessary Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import random
import cv2
from skimage.metrics import structural_similarity as ssim

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define paths to the original and test images
original_house = '/content/house.png'
original_clock = '/content/clock.png'

test_house = '/content/house_test.png'
test_clock = '/content/clock_test.png'

# Ensure the images exist
required_images = [original_house, original_clock, test_house, test_clock]
for img_path in required_images:
    if not os.path.exists(img_path):
        print(f"Error: {img_path} does not exist. Please upload the image to the specified path.")
        from google.colab import files
        files.upload()
        break

# Directory to save augmented images
augmented_dir = '/content/augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Parameters
img_width, img_height = 224, 224  # EfficientNetB0 input size
batch_size = 32
num_augmented = 850  # Number of augmented images per original image

# Create ImageDataGenerator instances for healthy and impaired augmentations
healthy_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.15,
    height_shift_range=0.15,
    zoom_range=0.15,
    shear_range=0.15,
    brightness_range=[0.8,1.2],
    horizontal_flip=True,
    fill_mode='nearest'
)

impaired_datagen = ImageDataGenerator(
    rotation_range=45,  # More rotation to simulate impairments
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.3,
    shear_range=0.3,
    brightness_range=[0.5,1.5],
    horizontal_flip=True,
    fill_mode='nearest'
)

# Function to augment images
def augment_image(image_path, label, datagen, save_dir, prefix, num_augmented):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    aug_iter = datagen.flow(
        x,
        batch_size=1,
        save_to_dir=save_dir,
        save_prefix=prefix,
        save_format='png'
    )

    labels = []
    for i in range(num_augmented):
        next(aug_iter)
        labels.append(label)
    return labels

# Create directories for healthy and impaired images
healthy_dir = os.path.join(augmented_dir, 'healthy')
impaired_dir = os.path.join(augmented_dir, 'impaired')
os.makedirs(healthy_dir, exist_ok=True)
os.makedirs(impaired_dir, exist_ok=True)

# Augment healthy images
labels_healthy = []
labels_healthy += augment_image(original_house, 0, healthy_datagen, healthy_dir, 'house_healthy', num_augmented)
labels_healthy += augment_image(original_clock, 0, healthy_datagen, healthy_dir, 'clock_healthy', num_augmented)

# Augment impaired images
labels_impaired = []
labels_impaired += augment_image(original_house, 1, impaired_datagen, impaired_dir, 'house_impaired', num_augmented)
labels_impaired += augment_image(original_clock, 1, impaired_datagen, impaired_dir, 'clock_impaired', num_augmented)

# Combine labels
labels = labels_healthy + labels_impaired

# Collect image file paths and corresponding labels
image_paths = []
for fname in os.listdir(healthy_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(healthy_dir, fname))
for fname in os.listdir(impaired_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(impaired_dir, fname))

# Shuffle the dataset
combined = list(zip(image_paths, labels))
random.shuffle(combined)
image_paths, labels = zip(*combined)

image_paths = np.array(image_paths)
labels = np.array(labels)

# Map numerical labels to string labels
label_mapping = {0: 'Healthy', 1: 'Impaired'}
labels_str = np.array([label_mapping[label] for label in labels])

# Create a dataframe with string labels
full_df = pd.DataFrame({'filename': image_paths, 'class': labels_str})

# Plot the distribution of classes
plt.figure(figsize=(6,4))
full_df['class'].value_counts().plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.show()

# Show some sample images
def show_sample_images(df, title):
    plt.figure(figsize=(12,6))
    samples = df.sample(5)
    for idx, row in enumerate(samples.iterrows()):
        img_path = row[1]['filename']
        img = load_img(img_path, target_size=(img_width, img_height))
        plt.subplot(1,5,idx+1)
        plt.imshow(img)
        plt.title(row[1]['class'])
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

show_sample_images(full_df[full_df['class'] == 'Healthy'], 'Sample Healthy Images')
show_sample_images(full_df[full_df['class'] == 'Impaired'], 'Sample Impaired Images')

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    full_df['filename'], full_df['class'], test_size=0.15, random_state=42, stratify=full_df['class'])

# Create separate dataframes for training and validation
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
val_df = pd.DataFrame({'filename': X_val, 'class': y_val})

print(f'Training samples: {len(train_df)}')
print(f'Validation samples: {len(val_df)}')

# Plot the distribution in training and validation sets
fig, axs = plt.subplots(1, 2, figsize=(12,4))

train_df['class'].value_counts().plot(kind='bar', ax=axs[0])
axs[0].set_title('Training Set Class Distribution')
axs[0].set_xlabel('Class')
axs[0].set_ylabel('Number of Samples')

val_df['class'].value_counts().plot(kind='bar', ax=axs[1])
axs[1].set_title('Validation Set Class Distribution')
axs[1].set_xlabel('Class')
axs[1].set_ylabel('Number of Samples')

plt.tight_layout()
plt.show()

# Define data generators with preprocessing
train_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

# Create generators
train_generator = train_datagen_flow.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen_flow.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)

# Build an Advanced CNN Model Using Transfer Learning (Functional API)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    '/content/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Train the model
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Fine-Tune the Model
base_model.trainable = True

# Freeze all layers except the top 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define the number of fine-tuning epochs
fine_tune_epochs = 10
initial_epoch = len(history.history['loss'])
total_epochs = initial_epoch + fine_tune_epochs

# Continue training with fine-tuning
history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=initial_epoch,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Load the best model saved during training
best_model = tf.keras.models.load_model('/content/best_model.keras')

# Function to preprocess and predict a single image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Batch dimension
    return img_array

def predict_image(image_path, model):
    preprocessed = preprocess_image(image_path)
    prediction = model.predict(preprocessed)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = 'Healthy' if class_idx == 0 else 'Impaired'
    confidence = prediction[0][class_idx]
    return class_label, confidence, prediction

# Define test images
test_images = [test_house, test_clock]

# Function to calculate similarity metrics
def calculate_similarity(original_path, test_path):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Convert to grayscale for SSIM
    original_gray = cv2.cvtColor(original, cv2.COLOR_RGB2GRAY)
    test_gray = cv2.cvtColor(test, cv2.COLOR_RGB2GRAY)

    # Calculate SSIM
    ssim_score = ssim(original_gray, test_gray)

    # Calculate MSE
    mse_score = np.mean((original - test) ** 2)

    return ssim_score, mse_score

# Function to visualize comparison
def visualize_comparison(original_path, test_path, title):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Calculate difference
    difference = cv2.absdiff(original, test)

    # Convert to grayscale
    difference_gray = cv2.cvtColor(difference, cv2.COLOR_RGB2GRAY)

    # Apply threshold to highlight differences
    _, thresh = cv2.threshold(difference_gray, 30, 255, cv2.THRESH_BINARY)

    # Calculate SSIM for display
    ssim_score, _ = calculate_similarity(original_path, test_path)

    # Plotting
    plt.figure(figsize=(15,5))

    plt.subplot(1,3,1)
    plt.imshow(original.astype('uint8'))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(test.astype('uint8'))
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(thresh, cmap='gray')
    plt.title(f'Difference Map\nSSIM: {ssim_score:.4f}')
    plt.axis('off')

    plt.suptitle(title, fontsize=16)
    plt.show()

# Function to generate Grad-CAM heatmap
def get_gradcam_heatmap(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Get the last convolutional layer
    last_conv_layer = model.get_layer('efficientnetb0').get_layer('top_conv')

    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = Model([model.inputs], [last_conv_layer.output, model.output])

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    # Compute gradients
    grads = tape.gradient(loss, conv_outputs)

    # Compute guided gradients
    guided_grads = grads[0]

    # Weigh the outputs of the conv layer with the gradients
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)

    # Apply ReLU to the heatmap
    heatmap = np.maximum(cam, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap.numpy()

def display_gradcam(image_path, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    # Load the original image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    # Superimpose the heatmap on the image
    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)

    # Convert BGR to RGB for displaying
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Function to generate analysis report
def generate_analysis_report(original_path, test_path, model):
    # Determine the drawing type
    if 'house' in test_path.lower():
        title = 'House Drawing Analysis'
    elif 'clock' in test_path.lower():
        title = 'Clock Drawing Analysis'
    else:
        title = 'Drawing Analysis'

    # Predict label and confidence
    label, conf, pred = predict_image(test_path, model)

    # Calculate similarity metrics
    ssim_score, mse_score = calculate_similarity(original_path, test_path)

    # Display similarity metrics
    print(f'--- {title} ---')
    print(f'Predicted Label: {label} with confidence {conf:.2f}')
    print(f'SSIM Score: {ssim_score:.4f}')
    print(f'MSE Score: {mse_score:.2f}')
    print('\n')

    # Visualize comparison
    visualize_comparison(original_path, test_path, title)

    # Generate Grad-CAM heatmap
    try:
        heatmap = get_gradcam_heatmap(model, test_path)
        print('Grad-CAM Heatmap:')
        display_gradcam(test_path, heatmap)
    except Exception as e:
        print(f"Error generating Grad-CAM: {e}")

    # Provide textual analysis
    if label == 'Impaired':
        print("Analysis: The drawing shows significant deviations from the original, indicating possible cognitive impairments associated with Alzheimer's disease.")
    else:
        print("Analysis: The drawing closely resembles the original, suggesting no significant cognitive impairments detected.")
    print('\n' + '='*80 + '\n')

# Generate analysis reports for test images
for test_img in test_images:
    if 'house' in test_img.lower():
        original_img = original_house
    elif 'clock' in test_img.lower():
        original_img = original_clock
    else:
        continue

    generate_analysis_report(original_img, test_img, best_model)

# Plot Training and Validation Metrics
def plot_training_history(history, fine_tune_history=None):
    plt.figure(figsize=(14,6))

    # Plot Accuracy
    plt.subplot(1,2,1)
    epochs_range = range(len(history.history['accuracy']))
    plt.plot(epochs_range, history.history['accuracy'], label='Train Accuracy')
    plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')

    if fine_tune_history:
        fine_tune_epochs_range = range(len(history.history['accuracy']), len(history.history['accuracy']) + len(fine_tune_history.history['accuracy']))
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['accuracy'], label='Fine-Tune Train Accuracy')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_accuracy'], label='Fine-Tune Validation Accuracy')

    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1,2,2)
    plt.plot(epochs_range, history.history['loss'], label='Train Loss')
    plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')

    if fine_tune_history:
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['loss'], label='Fine-Tune Train Loss')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_loss'], label='Fine-Tune Validation Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Plot the training history
plot_training_history(history, history_fine)


In [ ]:
# Mount Google Drive (optional, if you're saving or loading files from Google Drive)
from google.colab import drive
drive.mount('/content/drive')

# Import Necessary Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import random
import cv2
from skimage.metrics import structural_similarity as ssim

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define paths to the original and test images
original_house = '/content/house.png'
original_clock = '/content/clock.png'

test_house = '/content/house_test.png'
test_clock = '/content/clock_test.png'

# Ensure the images exist
required_images = [original_house, original_clock, test_house, test_clock]
for img_path in required_images:
    if not os.path.exists(img_path):
        print(f"Error: {img_path} does not exist. Please upload the image to the specified path.")
        from google.colab import files
        files.upload()
        break

# Directory to save augmented images
augmented_dir = '/content/augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Parameters
img_width, img_height = 224, 224  # EfficientNetB0 input size
num_augmented = 850  # Number of augmented images per original image

# Function to simulate impairment by removing or misplacing parts of the image
def impair_image(image_path, save_dir, prefix, num_augmented):
    img = Image.open(image_path).resize((img_width, img_height))
    labels = []
    for i in range(num_augmented):
        impaired_img = img.copy()
        draw = ImageDraw.Draw(impaired_img)

        # Decide how many parts to remove (e.g., 20% to 50% of the elements)
        num_parts_to_remove = random.randint(3, 7)

        for _ in range(num_parts_to_remove):
            # Randomly select a rectangle region to remove
            x1 = random.randint(0, img_width - 50)
            y1 = random.randint(0, img_height - 50)
            x2 = x1 + random.randint(20, 70)
            y2 = y1 + random.randint(20, 70)
            draw.rectangle([x1, y1, x2, y2], fill=(255, 255, 255))

        # Misplace elements by drawing random shapes
        num_parts_to_misplace = random.randint(1, 3)
        for _ in range(num_parts_to_misplace):
            shape_type = random.choice(['rectangle', 'ellipse'])
            x1 = random.randint(0, img_width - 50)
            y1 = random.randint(0, img_height - 50)
            x2 = x1 + random.randint(20, 70)
            y2 = y1 + random.randint(20, 70)
            if shape_type == 'rectangle':
                draw.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))
            else:
                draw.ellipse([x1, y1, x2, y2], fill=(0, 0, 0))

        # Save the impaired image
        impaired_img.save(os.path.join(save_dir, f"{prefix}_{i}.png"))
        labels.append(1)  # Label 1 for impaired
    return labels

# Function to simulate healthy images with slight augmentations
def healthy_image(image_path, datagen, save_dir, prefix, num_augmented):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Create an iterator for data augmentation
    aug_iter = datagen.flow(
        x,
        batch_size=1,
        save_to_dir=save_dir,
        save_prefix=prefix,
        save_format='png'
    )

    labels = []
    for i in range(num_augmented):
        next(aug_iter)
        labels.append(0)  # Label 0 for healthy
    return labels

# Create ImageDataGenerator instance for healthy images with slight augmentations
healthy_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    shear_range=0.05,
    brightness_range=[0.9,1.1],
    horizontal_flip=False,
    fill_mode='nearest'
)

# Create directories for healthy and impaired images
healthy_dir = os.path.join(augmented_dir, 'healthy')
impaired_dir = os.path.join(augmented_dir, 'impaired')
os.makedirs(healthy_dir, exist_ok=True)
os.makedirs(impaired_dir, exist_ok=True)

# Generate healthy images
labels_healthy = []
labels_healthy += healthy_image(original_house, healthy_datagen, healthy_dir, 'house_healthy', num_augmented)
labels_healthy += healthy_image(original_clock, healthy_datagen, healthy_dir, 'clock_healthy', num_augmented)

# Generate impaired images
labels_impaired = []
labels_impaired += impair_image(original_house, impaired_dir, 'house_impaired', num_augmented)
labels_impaired += impair_image(original_clock, impaired_dir, 'clock_impaired', num_augmented)

# Combine labels
labels = labels_healthy + labels_impaired

# Collect image file paths and corresponding labels
image_paths = []
for fname in os.listdir(healthy_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(healthy_dir, fname))
for fname in os.listdir(impaired_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(impaired_dir, fname))

# Shuffle the dataset
combined = list(zip(image_paths, labels))
random.shuffle(combined)
image_paths, labels = zip(*combined)

image_paths = np.array(image_paths)
labels = np.array(labels)

# Map numerical labels to string labels
label_mapping = {0: 'Healthy', 1: 'Impaired'}
labels_str = np.array([label_mapping[label] for label in labels])

# Create a dataframe with string labels
full_df = pd.DataFrame({'filename': image_paths, 'class': labels_str})

# Plot the distribution of classes
plt.figure(figsize=(6,4))
full_df['class'].value_counts().plot(kind='bar')
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.show()

# Show some sample images
def show_sample_images(df, title):
    plt.figure(figsize=(12,6))
    samples = df.sample(5)
    for idx, row in enumerate(samples.iterrows()):
        img_path = row[1]['filename']
        img = load_img(img_path, target_size=(img_width, img_height))
        plt.subplot(1,5,idx+1)
        plt.imshow(img)
        plt.title(row[1]['class'])
        plt.axis('off')
    plt.suptitle(title)
    plt.show()

show_sample_images(full_df[full_df['class'] == 'Healthy'], 'Sample Healthy Images')
show_sample_images(full_df[full_df['class'] == 'Impaired'], 'Sample Impaired Images')

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    full_df['filename'], full_df['class'], test_size=0.15, random_state=42, stratify=full_df['class'])

# Create separate dataframes for training and validation
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
val_df = pd.DataFrame({'filename': X_val, 'class': y_val})

print(f'Training samples: {len(train_df)}')
print(f'Validation samples: {len(val_df)}')

# Plot the distribution in training and validation sets
fig, axs = plt.subplots(1, 2, figsize=(12,4))

train_df['class'].value_counts().plot(kind='bar', ax=axs[0])
axs[0].set_title('Training Set Class Distribution')
axs[0].set_xlabel('Class')
axs[0].set_ylabel('Number of Samples')

val_df['class'].value_counts().plot(kind='bar', ax=axs[1])
axs[1].set_title('Validation Set Class Distribution')
axs[1].set_xlabel('Class')
axs[1].set_ylabel('Number of Samples')

plt.tight_layout()
plt.show()

# Define data generators with preprocessing
train_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

# Create generators
train_generator = train_datagen_flow.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen_flow.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)

# Build an Advanced CNN Model Using Transfer Learning (Functional API)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    '/content/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Train the model
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Fine-Tune the Model
base_model.trainable = True

# Freeze all layers except the top 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_epochs = 10
initial_epoch = len(history.history['loss'])
total_epochs = initial_epoch + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=initial_epoch,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Load the best model saved during training
best_model = tf.keras.models.load_model('/content/best_model.keras')

# Function to preprocess and predict a single image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Batch dimension
    return img_array

def predict_image(image_path, model):
    preprocessed = preprocess_image(image_path)
    prediction = model.predict(preprocessed)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = 'Healthy' if class_idx == 0 else 'Impaired'
    confidence = prediction[0][class_idx]
    return class_label, confidence, prediction

# Function to calculate missing parts percentage
def detect_missing_parts(original_path, test_path):
    # Load images in grayscale
    original = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    test = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE)

    # Resize images to the same size
    original = cv2.resize(original, (img_width, img_height))
    test = cv2.resize(test, (img_width, img_height))

    # Apply threshold to binarize images
    _, original_thresh = cv2.threshold(original, 127, 255, cv2.THRESH_BINARY_INV)
    _, test_thresh = cv2.threshold(test, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the images
    contours_original, _ = cv2.findContours(original_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_test, _ = cv2.findContours(test_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create blank images to draw contours
    original_contour_img = np.zeros_like(original_thresh)
    test_contour_img = np.zeros_like(test_thresh)

    # Draw contours
    cv2.drawContours(original_contour_img, contours_original, -1, (255, 255, 255), thickness=cv2.FILLED)
    cv2.drawContours(test_contour_img, contours_test, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Calculate the difference
    difference = cv2.absdiff(original_contour_img, test_contour_img)
    missing_pixels = cv2.countNonZero(difference)
    total_pixels = cv2.countNonZero(original_contour_img)

    # Calculate missing parts percentage
    if total_pixels == 0:
        total_pixels = 1e-5  # Prevent division by zero
    missing_percentage = (missing_pixels / total_pixels) * 100

    return missing_percentage

# Function to visualize comparison
def visualize_comparison(original_path, test_path, title):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Calculate difference
    difference = cv2.absdiff(original, test)

    # Convert to grayscale
    difference_gray = cv2.cvtColor(difference, cv2.COLOR_RGB2GRAY)

    # Apply threshold to highlight differences
    _, thresh = cv2.threshold(difference_gray, 30, 255, cv2.THRESH_BINARY)

    # Plotting
    plt.figure(figsize=(15,5))

    plt.subplot(1,3,1)
    plt.imshow(original.astype('uint8'))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1,3,2)
    plt.imshow(test.astype('uint8'))
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1,3,3)
    plt.imshow(thresh, cmap='gray')
    plt.title('Difference Map')
    plt.axis('off')

    plt.suptitle(title, fontsize=16)
    plt.show()

# Function to generate Grad-CAM heatmap
def get_gradcam_heatmap(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Get the last convolutional layer
    last_conv_layer = model.get_layer('efficientnetb0').get_layer('top_conv')

    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = Model([model.inputs], [last_conv_layer.output, model.output])

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    # Compute gradients
    grads = tape.gradient(loss, conv_outputs)

    # Compute guided gradients
    guided_grads = grads[0]

    # Weigh the outputs of the conv layer with the gradients
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)

    # Apply ReLU to the heatmap
    heatmap = np.maximum(cam, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap.numpy()

def display_gradcam(image_path, heatmap, alpha=0.4, colormap=cv2.COLORMAP_JET):
    # Load the original image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    # Superimpose the heatmap on the image
    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)

    # Convert BGR to RGB for displaying
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_img)
    plt.axis('off')
    plt.show()

# Function to generate analysis report
def generate_analysis_report(original_path, test_path, model):
    # Determine the drawing type
    if 'house' in test_path.lower():
        title = 'House Drawing Analysis'
    elif 'clock' in test_path.lower():
        title = 'Clock Drawing Analysis'
    else:
        title = 'Drawing Analysis'

    # Predict label and confidence
    label, conf, pred = predict_image(test_path, model)

    # Calculate missing parts percentage
    missing_percentage = detect_missing_parts(original_path, test_path)

    # Display analysis
    print(f'--- {title} ---')
    print(f'Predicted Label: {label} with confidence {conf:.2f}')
    print(f'Missing Parts Percentage: {missing_percentage:.2f}%')
    print('\n')

    # Visualize comparison
    visualize_comparison(original_path, test_path, title)

    # Generate Grad-CAM heatmap
    try:
        heatmap = get_gradcam_heatmap(model, test_path)
        print('Grad-CAM Heatmap:')
        display_gradcam(test_path, heatmap)
    except Exception as e:
        print(f"Error generating Grad-CAM: {e}")

    # Provide textual analysis
    if label == 'Impaired':
        print(f"Analysis: The drawing shows significant deviations with {missing_percentage:.2f}% missing or misplaced elements, indicating a higher possibility of cognitive impairments associated with Alzheimer's disease.")
    else:
        print(f"Analysis: The drawing closely resembles the original with {missing_percentage:.2f}% missing elements, suggesting no significant cognitive impairments detected.")
    print('\n' + '='*80 + '\n')

# Generate analysis reports for test images
test_images = [test_house, test_clock]
for test_img in test_images:
    if 'house' in test_img.lower():
        original_img = original_house
    elif 'clock' in test_img.lower():
        original_img = original_clock
    else:
        continue

    generate_analysis_report(original_img, test_img, best_model)

# Plot Training and Validation Metrics
def plot_training_history(history, fine_tune_history=None):
    plt.figure(figsize=(14,6))

    # Plot Accuracy
    plt.subplot(1,2,1)
    epochs_range = range(len(history.history['accuracy']))
    plt.plot(epochs_range, history.history['accuracy'], label='Train Accuracy')
    plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy')

    if fine_tune_history:
        fine_tune_epochs_range = range(len(history.history['accuracy']), len(history.history['accuracy']) + len(fine_tune_history.history['accuracy']))
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['accuracy'], label='Fine-Tune Train Accuracy')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_accuracy'], label='Fine-Tune Validation Accuracy')

    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    # Plot Loss
    plt.subplot(1,2,2)
    plt.plot(epochs_range, history.history['loss'], label='Train Loss')
    plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss')

    if fine_tune_history:
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['loss'], label='Fine-Tune Train Loss')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_loss'], label='Fine-Tune Validation Loss')

    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

# Plot the training history
plot_training_history(history, history_fine)


In [ ]:
# Import Necessary Libraries
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import pandas as pd
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import tensorflow as tf
import random
import cv2
from skimage.metrics import structural_similarity as ssim

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

# Define paths to the original and test images
original_house = '/content/house.png'
original_clock = '/content/clock.png'

test_house = '/content/house_test.png'
test_clock = '/content/clock_test.png'

# Ensure the images exist
required_images = [original_house, original_clock, test_house, test_clock]
for img_path in required_images:
    if not os.path.exists(img_path):
        print(f"Error: {img_path} does not exist. Please upload the image to the specified path.")
        from google.colab import files
        files.upload()
        break

# Directory to save augmented images
augmented_dir = '/content/augmented'
os.makedirs(augmented_dir, exist_ok=True)

# Parameters
img_width, img_height = 224, 224  # EfficientNetB0 input size
num_augmented = 850  # Number of augmented images per original image

# Function to simulate impairment by removing or misplacing parts of the image
def impair_image(image_path, save_dir, prefix, num_augmented):
    img = Image.open(image_path).resize((img_width, img_height))
    labels = []
    for i in range(num_augmented):
        impaired_img = img.copy()
        draw = ImageDraw.Draw(impaired_img)

        # Decide how many parts to remove (e.g., 20% to 50% of the elements)
        num_parts_to_remove = random.randint(3, 7)

        for _ in range(num_parts_to_remove):
            # Randomly select a rectangle region to remove
            x1 = random.randint(0, img_width - 50)
            y1 = random.randint(0, img_height - 50)
            x2 = x1 + random.randint(20, 70)
            y2 = y1 + random.randint(20, 70)
            draw.rectangle([x1, y1, x2, y2], fill=(255, 255, 255))

        # Misplace elements by drawing random shapes
        num_parts_to_misplace = random.randint(1, 3)
        for _ in range(num_parts_to_misplace):
            shape_type = random.choice(['rectangle', 'ellipse'])
            x1 = random.randint(0, img_width - 50)
            y1 = random.randint(0, img_height - 50)
            x2 = x1 + random.randint(20, 70)
            y2 = y1 + random.randint(20, 70)
            if shape_type == 'rectangle':
                draw.rectangle([x1, y1, x2, y2], fill=(0, 0, 0))
            else:
                draw.ellipse([x1, y1, x2, y2], fill=(0, 0, 0))

        # Save the impaired image
        impaired_img.save(os.path.join(save_dir, f"{prefix}_{i}.png"))
        labels.append(1)  # Label 1 for impaired
    return labels

# Function to simulate healthy images with slight augmentations
def healthy_image(image_path, datagen, save_dir, prefix, num_augmented):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    x = img_to_array(img)
    x = np.expand_dims(x, axis=0)

    # Create an iterator for data augmentation
    aug_iter = datagen.flow(
        x,
        batch_size=1,
        save_to_dir=save_dir,
        save_prefix=prefix,
        save_format='png'
    )

    labels = []
    for i in range(num_augmented):
        next(aug_iter)
        labels.append(0)  # Label 0 for healthy
    return labels

# Create ImageDataGenerator instance for healthy images with slight augmentations
healthy_datagen = ImageDataGenerator(
    rotation_range=5,
    width_shift_range=0.05,
    height_shift_range=0.05,
    zoom_range=0.05,
    shear_range=0.05,
    brightness_range=[0.9,1.1],
    horizontal_flip=False,
    fill_mode='nearest'
)

# Create directories for healthy and impaired images
healthy_dir = os.path.join(augmented_dir, 'healthy')
impaired_dir = os.path.join(augmented_dir, 'impaired')
os.makedirs(healthy_dir, exist_ok=True)
os.makedirs(impaired_dir, exist_ok=True)

# Generate healthy images
labels_healthy = []
labels_healthy += healthy_image(original_house, healthy_datagen, healthy_dir, 'house_healthy', num_augmented)
labels_healthy += healthy_image(original_clock, healthy_datagen, healthy_dir, 'clock_healthy', num_augmented)

# Generate impaired images
labels_impaired = []
labels_impaired += impair_image(original_house, impaired_dir, 'house_impaired', num_augmented)
labels_impaired += impair_image(original_clock, impaired_dir, 'clock_impaired', num_augmented)

# Combine labels
labels = labels_healthy + labels_impaired

# Collect image file paths and corresponding labels
image_paths = []
for fname in os.listdir(healthy_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(healthy_dir, fname))
for fname in os.listdir(impaired_dir):
    if fname.endswith('.png'):
        image_paths.append(os.path.join(impaired_dir, fname))

# Shuffle the dataset
combined = list(zip(image_paths, labels))
random.shuffle(combined)
image_paths, labels = zip(*combined)

image_paths = np.array(image_paths)
labels = np.array(labels)

# Map numerical labels to string labels
label_mapping = {0: 'Healthy', 1: 'Impaired'}
labels_str = np.array([label_mapping[label] for label in labels])

# Create a dataframe with string labels
full_df = pd.DataFrame({'filename': image_paths, 'class': labels_str})

# Plot the distribution of classes
plt.figure(figsize=(6,4))
full_df['class'].value_counts().plot(kind='bar', color=['green', 'red'])
plt.title('Class Distribution')
plt.xlabel('Class')
plt.ylabel('Number of Samples')
plt.xticks(rotation=0)
plt.show()

# Show some sample images
def show_sample_images(df, title):
    plt.figure(figsize=(15,6))
    samples = df.sample(5)
    for idx, row in enumerate(samples.iterrows()):
        img_path = row[1]['filename']
        img = load_img(img_path, target_size=(img_width, img_height))
        plt.subplot(1,5,idx+1)
        plt.imshow(img)
        plt.title(row[1]['class'])
        plt.axis('off')
    plt.suptitle(title, fontsize=16)
    plt.show()

show_sample_images(full_df[full_df['class'] == 'Healthy'], 'Sample Healthy Images')
show_sample_images(full_df[full_df['class'] == 'Impaired'], 'Sample Impaired Images')

# Split into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    full_df['filename'], full_df['class'], test_size=0.15, random_state=42, stratify=full_df['class'])

# Create separate dataframes for training and validation
train_df = pd.DataFrame({'filename': X_train, 'class': y_train})
val_df = pd.DataFrame({'filename': X_val, 'class': y_val})

print(f'Training samples: {len(train_df)}')
print(f'Validation samples: {len(val_df)}')

# Plot the distribution in training and validation sets
fig, axs = plt.subplots(1, 2, figsize=(12,4))

train_df['class'].value_counts().plot(kind='bar', ax=axs[0], color=['green', 'red'])
axs[0].set_title('Training Set Class Distribution')
axs[0].set_xlabel('Class')
axs[0].set_ylabel('Number of Samples')
axs[0].set_xticklabels(axs[0].get_xticklabels(), rotation=0)

val_df['class'].value_counts().plot(kind='bar', ax=axs[1], color=['green', 'red'])
axs[1].set_title('Validation Set Class Distribution')
axs[1].set_xlabel('Class')
axs[1].set_ylabel('Number of Samples')
axs[1].set_xticklabels(axs[1].get_xticklabels(), rotation=0)

plt.tight_layout()
plt.show()

# Define data generators with preprocessing
train_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)
val_datagen_flow = ImageDataGenerator(preprocessing_function=tf.keras.applications.efficientnet.preprocess_input)

# Create generators
train_generator = train_datagen_flow.flow_from_dataframe(
    dataframe=train_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

validation_generator = val_datagen_flow.flow_from_dataframe(
    dataframe=val_df,
    x_col='filename',
    y_col='class',
    target_size=(img_width, img_height),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

print("Class indices:", train_generator.class_indices)

# Build an Advanced CNN Model Using Transfer Learning (Functional API)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))
base_model.trainable = False

inputs = tf.keras.Input(shape=(img_width, img_height, 3))
x = base_model(inputs, training=False)
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(2, activation='softmax')(x)

model = Model(inputs, outputs)

model.compile(optimizer=Adam(learning_rate=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(
    '/content/best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    save_weights_only=False,
    mode='min',
    verbose=1
)

# Train the model
epochs = 20

history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Fine-Tune the Model
base_model.trainable = True

# Freeze all layers except the top 20 layers
for layer in base_model.layers[:-20]:
    layer.trainable = False

model.compile(optimizer=Adam(learning_rate=1e-5),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Continue training with fine-tuning
fine_tune_epochs = 10
initial_epoch = len(history.history['loss'])
total_epochs = initial_epoch + fine_tune_epochs

history_fine = model.fit(
    train_generator,
    epochs=total_epochs,
    initial_epoch=initial_epoch,
    validation_data=validation_generator,
    callbacks=[early_stop, checkpoint]
)

# Load the best model saved during training
best_model = tf.keras.models.load_model('/content/best_model.keras')

# Function to preprocess and predict a single image
def preprocess_image(image_path):
    img = load_img(image_path, color_mode='rgb', target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)  # Batch dimension
    return img_array

def predict_image(image_path, model):
    preprocessed = preprocess_image(image_path)
    prediction = model.predict(preprocessed)
    class_idx = np.argmax(prediction, axis=1)[0]
    class_label = 'Healthy' if class_idx == 0 else 'Impaired'
    confidence = prediction[0][class_idx]
    return class_label, confidence, prediction

# Function to calculate missing parts percentage
def detect_missing_parts(original_path, test_path):
    # Load images in grayscale
    original = cv2.imread(original_path, cv2.IMREAD_GRAYSCALE)
    test = cv2.imread(test_path, cv2.IMREAD_GRAYSCALE)

    # Resize images to the same size
    original = cv2.resize(original, (img_width, img_height))
    test = cv2.resize(test, (img_width, img_height))

    # Apply threshold to binarize images
    _, original_thresh = cv2.threshold(original, 127, 255, cv2.THRESH_BINARY_INV)
    _, test_thresh = cv2.threshold(test, 127, 255, cv2.THRESH_BINARY_INV)

    # Find contours in the images
    contours_original, _ = cv2.findContours(original_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_test, _ = cv2.findContours(test_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create blank images to draw contours
    original_contour_img = np.zeros_like(original_thresh)
    test_contour_img = np.zeros_like(test_thresh)

    # Draw contours
    cv2.drawContours(original_contour_img, contours_original, -1, (255, 255, 255), thickness=cv2.FILLED)
    cv2.drawContours(test_contour_img, contours_test, -1, (255, 255, 255), thickness=cv2.FILLED)

    # Calculate the difference
    difference = cv2.absdiff(original_contour_img, test_contour_img)
    missing_pixels = cv2.countNonZero(difference)
    total_pixels = cv2.countNonZero(original_contour_img)

    # Calculate missing parts percentage
    if total_pixels == 0:
        total_pixels = 1e-5  # Prevent division by zero
    missing_percentage = (missing_pixels / total_pixels) * 100

    return missing_percentage

# Function to visualize comparison
def visualize_comparison(original_path, test_path, title):
    original = load_img(original_path, color_mode='rgb', target_size=(img_width, img_height))
    test = load_img(test_path, color_mode='rgb', target_size=(img_width, img_height))

    original = img_to_array(original).astype('uint8')
    test = img_to_array(test).astype('uint8')

    # Calculate difference
    difference = cv2.absdiff(original, test)

    # Convert to grayscale
    difference_gray = cv2.cvtColor(difference, cv2.COLOR_RGB2GRAY)

    # Apply threshold to highlight differences
    _, thresh = cv2.threshold(difference_gray, 30, 255, cv2.THRESH_BINARY)

    # Plotting
    plt.figure(figsize=(20,6))

    plt.subplot(1,4,1)
    plt.imshow(original.astype('uint8'))
    plt.title('Original Image')
    plt.axis('off')

    plt.subplot(1,4,2)
    plt.imshow(test.astype('uint8'))
    plt.title('Test Image')
    plt.axis('off')

    plt.subplot(1,4,3)
    plt.imshow(thresh, cmap='gray')
    plt.title('Difference Map')
    plt.axis('off')

    # Overlay difference on test image
    overlay = test.copy()
    overlay[thresh > 0] = [255, 0, 0]  # Highlight differences in red

    plt.subplot(1,4,4)
    plt.imshow(overlay.astype('uint8'))
    plt.title('Highlighted Differences')
    plt.axis('off')

    plt.suptitle(title, fontsize=18)
    plt.show()

# Function to generate Grad-CAM heatmap
def get_gradcam_heatmap(model, image_path):
    # Load and preprocess the image
    img = load_img(image_path, target_size=(img_width, img_height))
    img_array = img_to_array(img)
    img_array = tf.keras.applications.efficientnet.preprocess_input(img_array)
    img_array = np.expand_dims(img_array, axis=0)

    # Get the last convolutional layer
    last_conv_layer = model.get_layer('efficientnetb0').get_layer('top_conv')

    # Create a model that maps the input image to the activations of the last conv layer
    grad_model = Model([model.inputs], [last_conv_layer.output, model.output])

    # Compute the gradient of the top predicted class for the input image
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        predicted_class = tf.argmax(predictions[0])
        loss = predictions[:, predicted_class]

    # Compute gradients
    grads = tape.gradient(loss, conv_outputs)

    # Compute guided gradients
    guided_grads = grads[0]

    # Weigh the outputs of the conv layer with the gradients
    weights = tf.reduce_mean(guided_grads, axis=(0, 1))
    cam = tf.reduce_sum(tf.multiply(weights, conv_outputs[0]), axis=-1)

    # Apply ReLU to the heatmap
    heatmap = np.maximum(cam, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap.numpy()

def display_gradcam(image_path, heatmap, alpha=0.6, colormap=cv2.COLORMAP_JET):
    # Load the original image
    img = cv2.imread(image_path)
    img = cv2.resize(img, (img_width, img_height))

    # Resize heatmap to match the image size
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))

    # Convert to RGB
    heatmap = np.uint8(255 * heatmap)
    heatmap_colored = cv2.applyColorMap(heatmap, colormap)

    # Superimpose the heatmap on the image
    superimposed_img = cv2.addWeighted(heatmap_colored, alpha, img, 1 - alpha, 0)

    # Convert BGR to RGB for displaying
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    plt.figure(figsize=(8,6))
    plt.imshow(superimposed_img)
    plt.title('Grad-CAM Heatmap')
    plt.axis('off')
    plt.show()

# Function to generate analysis report
def generate_analysis_report(original_path, test_path, model):
    # Determine the drawing type
    if 'house' in test_path.lower():
        title = 'House Drawing Analysis'
    elif 'clock' in test_path.lower():
        title = 'Clock Drawing Analysis'
    else:
        title = 'Drawing Analysis'

    # Predict label and confidence
    label, conf, pred = predict_image(test_path, model)

    # Calculate missing parts percentage
    missing_percentage = detect_missing_parts(original_path, test_path)

    # Display analysis
    print(f'--- {title} ---')
    print(f'Predicted Label: {label} (Confidence: {conf*100:.2f}%)')
    print(f'Missing Parts Percentage: {missing_percentage:.2f}%')
    print('\nDetailed Analysis:')

    if missing_percentage < 10:
        severity = 'low'
    elif missing_percentage < 30:
        severity = 'moderate'
    else:
        severity = 'high'

    print(f"- The drawing has a {severity} level of missing or misplaced elements.")
    print(f"- {missing_percentage:.2f}% of the original drawing elements are missing or altered.")
    print(f"- This indicates a {severity} possibility of cognitive impairments associated with Alzheimer's disease.")

    print('\nVisual Comparisons:')
    visualize_comparison(original_path, test_path, title)

    # Generate Grad-CAM heatmap
    try:
        heatmap = get_gradcam_heatmap(model, test_path)
        display_gradcam(test_path, heatmap)
    except Exception as e:
        print(f"Error generating Grad-CAM: {e}")

    print('\n' + '='*100 + '\n')

# Generate analysis reports for test images
test_images = [test_house, test_clock]
for test_img in test_images:
    if 'house' in test_img.lower():
        original_img = original_house
    elif 'clock' in test_img.lower():
        original_img = original_clock
    else:
        continue

    generate_analysis_report(original_img, test_img, best_model)

# Plot Training and Validation Metrics
def plot_training_history(history, fine_tune_history=None):
    plt.figure(figsize=(14,6))

    # Plot Accuracy
    plt.subplot(1,2,1)
    epochs_range = range(len(history.history['accuracy']))
    plt.plot(epochs_range, history.history['accuracy'], label='Train Accuracy', color='blue')
    plt.plot(epochs_range, history.history['val_accuracy'], label='Validation Accuracy', color='orange')

    if fine_tune_history:
        fine_tune_epochs_range = range(len(history.history['accuracy']), len(history.history['accuracy']) + len(fine_tune_history.history['accuracy']))
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['accuracy'], label='Fine-Tune Train Accuracy', color='green')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_accuracy'], label='Fine-Tune Validation Accuracy', color='red')

    plt.title('Training and Validation Accuracy', fontsize=16)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Accuracy', fontsize=14)
    plt.legend()
    plt.grid(True)

    # Plot Loss
    plt.subplot(1,2,2)
    plt.plot(epochs_range, history.history['loss'], label='Train Loss', color='blue')
    plt.plot(epochs_range, history.history['val_loss'], label='Validation Loss', color='orange')

    if fine_tune_history:
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['loss'], label='Fine-Tune Train Loss', color='green')
        plt.plot(fine_tune_epochs_range, fine_tune_history.history['val_loss'], label='Fine-Tune Validation Loss', color='red')

    plt.title('Training and Validation Loss', fontsize=16)
    plt.xlabel('Epoch', fontsize=14)
    plt.ylabel('Loss', fontsize=14)
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.show()

# Plot the training history
plot_training_history(history, history_fine)
